In [12]:
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import pandas as pd
import cPickle as pickle
import numpy as np
import os
app_path = '/Users/nmouterde/Documents/projectsPython/steam-game-engine/'

In [41]:
data = pickle.load(open(os.path.join(app_path, 'data_processed/tf_idf_data.p'), 'rb'))
names_tfidf = pickle.load(open(os.path.join(app_path, 'data_processed/names_tf_idf.p'), 'rb'))
names_tfidf = np.array(names_tfidf)

In [11]:
glove2word2vec(os.path.join(app_path, 'models/glove.twitter.27B.200d.txt'),
                            os.path.join(app_path, 'models/glove_w2v_200'))

(1193514, 200)

In [13]:
model = KeyedVectors.load_word2vec_format(os.path.join(app_path, 'models/glove_w2v_200'))

In [70]:
def construct_vector(row):
    coeff = 0
    vector = np.zeros(200)
    for w in row['cat1']:
        if w in model:
            vector = vector + model[w]*1.5
            coeff += 1.5
    for w in row['cat2']:
        if w in model:
            vector = vector + model[w]
            coeff += 1
    for w in row['cat3']:
        if (w in model) and (w in names_tfidf):
            idx_tfidf = np.where(names_tfidf == w)[0][0]
            tfidf_score = row['tfidf_score'][idx_tfidf]
            c = tfidf_score*0.5
            coeff+=c
            vector = vector + model[w]*c
        
    vector = vector/coeff
    return vector.tolist()

In [75]:
data['vec'] = data.apply(lambda x: construct_vector(x), axis=1)

In [98]:
write_line = data.apply(lambda x: str(x['id']) + ' ' + 
                        str(x['vec']).replace(',','')
                        .replace('[', '')
                        .replace(']', ''),
                        axis=1)

In [106]:
fichier = open(os.path.join(app_path, "models/gamevectors"), "w") 

In [107]:
for line in write_line.values:
    fichier.write(line+'\n')


In [110]:
fichier.close()
glove2word2vec(os.path.join(app_path, "models/gamevectors"),
               os.path.join(app_path, "models/gamevectors_w2c"))

(2, 200)

In [111]:
model2 = KeyedVectors.load_word2vec_format(os.path.join(app_path, 'models/gamevectors_w2c'))

In [120]:
vector = model['king']
model2.most_similar([vector])

[(u'20', 0.5311635136604309), (u'10', 0.46505841612815857)]